# Passage Reranking with Permutation Self-Consistency

This notebook implements the passage reranking experiment from the paper "Found in the Middle: Permutation Self-Consistency Improves Listwise Ranking in Large Language Models" (arXiv:2310.07712).

We use MS MARCO collection with TREC DL19/DL20 evaluation sets.


**Global Inputs**
- Set your OpenAI API key here. If you're using Azure, see the code documentation for `OpenAIConfig` for how to modify it.
- Set the aggregate size here. The paper uses 20 permutations for passage reranking.
- Choose which TREC track to evaluate (DL19 or DL20).


In [1]:
api_key = ''
api_type = 'openai'  # or 'azure'
num_aggregates = 20  # number of permutations (paper uses 20 for passage reranking)
num_limit = 10  # number of queries to process (set to 200 for full DL19/DL20; 10 for testing)
track = 'dl19'  # 'dl19' or 'dl20'

# Force reload permsc module to pick up code changes
import importlib
import sys
if 'permsc' in sys.modules:
    importlib.reload(sys.modules['permsc'])
if 'permsc.data' in sys.modules:
    importlib.reload(sys.modules['permsc.data'])
if 'permsc.aggregator' in sys.modules:
    importlib.reload(sys.modules['permsc.aggregator'])
if 'permsc.aggregator.exact' in sys.modules:
    importlib.reload(sys.modules['permsc.aggregator.exact'])
if 'permsc.aggregator.utils' in sys.modules:
    importlib.reload(sys.modules['permsc.aggregator.utils'])
if 'permsc.evaluation' in sys.modules:
    importlib.reload(sys.modules['permsc.evaluation'])


In [2]:
import multiprocessing as mp

# Fix for multiprocessing in Jupyter notebooks on macOS
# Set start method to 'fork' if available, otherwise 'spawn'
try:
    mp.set_start_method('fork', force=True)
except RuntimeError:
    # Already set, or 'fork' not available (Windows)
    pass

from permsc import (
    RelevanceRankingPromptBuilder,
    OpenAIPromptPipeline,
    OpenAIConfig,
    ChatCompletionPool,
    KemenyOptimalAggregator,
    MSMarcoDataset,
    ndcg_at_k,
    mrr_at_k
)

config = OpenAIConfig(model_name='gpt-3.5-turbo', api_key=api_key, api_type=api_type)
builder = RelevanceRankingPromptBuilder()
pool = ChatCompletionPool([config] * 5)  # 5 parallel instances
pipeline = OpenAIPromptPipeline(builder, pool)


In [3]:
# Load dataset based on selected track
if track == 'dl19':
    ds = MSMarcoDataset(
        collection_path='../data/msmarco/collection.tsv',
        queries_path='../data/trec-dl19/msmarco-test2019-queries.tsv',
        qrels_path='../data/trec-dl19/2019qrels-pass.txt',
        top1000_path='../data/trec-dl19/msmarco-passagetest2019-top1000.tsv',
        max_passages=100
    )
elif track == 'dl20':
    ds = MSMarcoDataset(
        collection_path='../data/msmarco/collection.tsv',
        queries_path='../data/trec-dl20/msmarco-test2020-queries.tsv',
        qrels_path='../data/trec-dl20/2020qrels-pass.txt',
        top1000_path='../data/trec-dl20/msmarco-passagetest2020-top1000.tsv',
        max_passages=100
    )
else:
    raise ValueError(f"Unknown track: {track}")

print(f"Dataset loaded: {len(ds)} queries")
print(f"First query: {ds[0].query.content[:80]}...")
print(f"First query has {len(ds[0].hits)} passages")


Dataset loaded: 200 queries
First query: which amendment protects a person from cruel or unusual punishment...
First query has 100 passages


In [4]:
from copy import deepcopy
import numpy as np


def run_passage_reranking_pipeline(pipeline, dataset, num_aggregates, limit=100):
    """
    Run permutation self-consistency pipeline for passage reranking.
    
    Returns:
        prefs_list: List of preference arrays (one per query, each with num_aggregates permutations)
        perms_list: List of permutation arrays (input permutations used)
        qrels_dict: Dictionary mapping query_id to qrels dict
    """
    prefs_list = []
    perms_list = []
    qrels_dict = {}
    
    for example in dataset[:limit]:
        example = deepcopy(example)
        query_id = example.metadata.get('query_id')
        qrels_dict[query_id] = example.metadata.get('qrels', {})
        
        prefs = []
        items = []
        perms = []
        
        # Generate num_aggregates permutations
        for _ in range(num_aggregates):
            ex_cpy = deepcopy(example)
            perm = ex_cpy.randomize_order()
            perms.append(perm)
            items.append(ex_cpy)
        
        # Run pipeline
        outputs = pipeline.run(items, temperature=0, request_timeout=30)
        
        # Restore preferences to original order
        for output, perm_example in zip(outputs, items):
            # Use the permuted example's restore method
            restored_prefs = perm_example.permuted_preferences_to_original_order(output)
            # CRITICAL: Convert to int64 to avoid Numba typing errors
            if restored_prefs.dtype != np.int64 and restored_prefs.dtype != np.int32:
                restored_prefs = restored_prefs.astype(np.int64)
            prefs.append(restored_prefs)
        
        # Ensure the array is int64
        prefs_array = np.array(prefs, dtype=np.int64)
        prefs_list.append(prefs_array)
        perms_list.append(np.array(perms))
    
    return prefs_list, perms_list, qrels_dict


In [5]:
def aggregate_rankings(prefs_list):
    """Aggregate multiple preference rankings using Kemeny optimal aggregation."""
    import numpy as np
    aggregator = KemenyOptimalAggregator()
    results = []
    
    for prefs in prefs_list:
        # CRITICAL: Convert to int64 BEFORE aggregation to avoid Numba typing errors
        if not (prefs.dtype == np.int64 or prefs.dtype == np.int32):
            prefs = prefs.astype(np.int64)
        aggregated = aggregator.aggregate(prefs)
        results.append(aggregated)
    
    return results


In [6]:
def evaluate_reranking(results, dataset, qrels_dict):
    """
    Evaluate aggregated rankings using nDCG@10 and MRR@10.
    
    Returns:
        Dictionary with mean metrics and per-query scores
    """
    ndcg_scores = []
    mrr_scores = []
    
    for idx, (result, example) in enumerate(zip(results, dataset)):
        query_id = example.metadata.get('query_id')
        qrels = qrels_dict.get(query_id, {})
        
        # Convert result to ranked passage IDs
        # Handle both indices (integers) and passage IDs (strings)
        if len(result) > 0:
            # Check if first element is an integer (index) or string (passage ID)
            first_elem = result[0] if not isinstance(result, np.ndarray) else result.flat[0]
            if isinstance(first_elem, (int, np.integer)):
                # Result contains indices - convert to passage IDs
                ranked_passage_ids = [example.hits[i].id for i in result if i != -1 and i < len(example.hits)]
            else:
                # Result contains passage IDs directly
                ranked_passage_ids = [pid for pid in result if pid]
        else:
            ranked_passage_ids = []
        
        # Compute metrics
        ndcg = ndcg_at_k(ranked_passage_ids, qrels, k=10)
        mrr = mrr_at_k(ranked_passage_ids, qrels, k=10)
        
        ndcg_scores.append(ndcg)
        mrr_scores.append(mrr)
    
    return {
        'ndcg@10': np.mean(ndcg_scores),
        'mrr@10': np.mean(mrr_scores),
        'ndcg_scores': ndcg_scores,
        'mrr_scores': mrr_scores
    }


## Run Permutation Self-Consistency Pipeline


In [7]:
# Run pipeline
print(f"Running pipeline on {num_limit} queries with {num_aggregates} permutations each...")
prefs_list, perms_list, qrels_dict = run_passage_reranking_pipeline(
    pipeline, ds, num_aggregates, limit=num_limit
)
print(f"Completed {len(prefs_list)} queries")


Running pipeline on 10 queries with 20 permutations each...


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

Completed 10 queries


In [8]:
# Aggregate rankings
print("Aggregating rankings...")
results = aggregate_rankings(prefs_list)
print(f"Aggregated {len(results)} rankings")


Aggregating rankings...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/arul/Documents/diff_psc/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/lv/3b31xxlx2vg_ntbjdr6b2rtm0000gn/T/27f8c5f69e0e4c2ea75f92c1ec2155ef-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/lv/3b31xxlx2vg_ntbjdr6b2rtm0000gn/T/27f8c5f69e0e4c2ea75f92c1ec2155ef-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5 COLUMNS
At line 7 RHS
At line 8 BOUNDS
At line 10 ENDATA
Problem MODEL has 0 rows, 1 columns and 0 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Empty problem - 0 rows, 1 columns and 0 elements
Optimal - objective value 0
Optimal objective 0 - 0 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

Welcome to the CBC MILP Solver 
Version:

In [9]:
# Evaluate aggregated results
metrics = evaluate_reranking(results, ds[:num_limit], qrels_dict)
print(f"\n=== Aggregated Results (PSC) ===")
print(f"nDCG@10: {metrics['ndcg@10']:.4f}")
print(f"MRR@10: {metrics['mrr@10']:.4f}")



=== Aggregated Results (PSC) ===
nDCG@10: 0.0000
MRR@10: 0.0000


## Compare with Baseline (First-Stage Retrieval)


In [10]:
# Evaluate baseline (original retrieval order)
baseline_results = []
for example in ds[:num_limit]:
    # Use original retrieval order (passages are already in retrieval order)
    # Use indices [0, 1, 2, ...] to represent original order
    ranked_indices = list(range(len(example.hits)))
    baseline_results.append(ranked_indices)

baseline_metrics = evaluate_reranking(baseline_results, ds[:num_limit], qrels_dict)
print(f"\n=== Baseline (First-Stage Retrieval) ===")
print(f"nDCG@10: {baseline_metrics['ndcg@10']:.4f}")
print(f"MRR@10: {baseline_metrics['mrr@10']:.4f}")

print(f"\n=== Improvement ===")
print(f"nDCG@10 improvement: {metrics['ndcg@10'] - baseline_metrics['ndcg@10']:.4f} ({((metrics['ndcg@10'] / baseline_metrics['ndcg@10'] - 1) * 100):.2f}%)")
print(f"MRR@10 improvement: {metrics['mrr@10'] - baseline_metrics['mrr@10']:.4f} ({((metrics['mrr@10'] / baseline_metrics['mrr@10'] - 1) * 100):.2f}%)")



=== Baseline (First-Stage Retrieval) ===
nDCG@10: 0.0000
MRR@10: 0.0000

=== Improvement ===
nDCG@10 improvement: 0.0000 (nan%)
MRR@10 improvement: 0.0000 (nan%)


/var/folders/lv/3b31xxlx2vg_ntbjdr6b2rtm0000gn/T/ipykernel_81765/2237042895.py:15: RuntimeWarning: invalid value encountered in scalar divide
  print(f"nDCG@10 improvement: {metrics['ndcg@10'] - baseline_metrics['ndcg@10']:.4f} ({((metrics['ndcg@10'] / baseline_metrics['ndcg@10'] - 1) * 100):.2f}%)")
/var/folders/lv/3b31xxlx2vg_ntbjdr6b2rtm0000gn/T/ipykernel_81765/2237042895.py:16: RuntimeWarning: invalid value encountered in scalar divide
  print(f"MRR@10 improvement: {metrics['mrr@10'] - baseline_metrics['mrr@10']:.4f} ({((metrics['mrr@10'] / baseline_metrics['mrr@10'] - 1) * 100):.2f}%)")


## Individual Run Performance (without aggregation)

Compare individual permutation runs to see the benefit of aggregation.


In [11]:
def evaluate_individual_runs(prefs_list, dataset, qrels_dict):
    """Evaluate each individual permutation run."""
    num_runs = len(prefs_list[0]) if prefs_list else 0
    individual_ndcg = [[] for _ in range(num_runs)]
    individual_mrr = [[] for _ in range(num_runs)]
    
    for idx, (prefs, example) in enumerate(zip(prefs_list, dataset)):
        query_id = example.metadata.get('query_id')
        qrels = qrels_dict.get(query_id, {})
        
        for run_idx, pref in enumerate(prefs):
            # Convert preference array to ranked passage IDs
            ranked_passage_ids = [example.hits[i].id for i in pref if i != -1]
            
            ndcg = ndcg_at_k(ranked_passage_ids, qrels, k=10)
            mrr = mrr_at_k(ranked_passage_ids, qrels, k=10)
            
            individual_ndcg[run_idx].append(ndcg)
            individual_mrr[run_idx].append(mrr)
    
    return {
        'ndcg': [np.mean(scores) for scores in individual_ndcg],
        'mrr': [np.mean(scores) for scores in individual_mrr]
    }

individual_metrics = evaluate_individual_runs(prefs_list, ds[:num_limit], qrels_dict)
print(f"Individual runs nDCG@10: {np.mean(individual_metrics['ndcg']):.4f} ± {np.std(individual_metrics['ndcg']):.4f}")
print(f"Individual runs MRR@10: {np.mean(individual_metrics['mrr']):.4f} ± {np.std(individual_metrics['mrr']):.4f}")
print(f"\nAggregated nDCG@10: {metrics['ndcg@10']:.4f}")
print(f"Aggregated MRR@10: {metrics['mrr@10']:.4f}")


Individual runs nDCG@10: nan ± nan
Individual runs MRR@10: nan ± nan

Aggregated nDCG@10: 0.0000
Aggregated MRR@10: 0.0000


/Users/arul/Documents/diff_psc/venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/arul/Documents/diff_psc/venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/arul/Documents/diff_psc/venv/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/arul/Documents/diff_psc/venv/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/arul/Documents/diff_psc/venv/lib/python3.10/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
